## Motifs and features

In [22]:
class Features:
    """ Features: features of the motif -
        pattern - sequence
        rank - rank/priority/id (must be unique)
        valid - valid motif or not (only for universal frequency)
    """

    __slots__ = ('pattern', 'rank', 'valid')
    pattern: str
    rank: int
    valid: bool

    def __init__(self, pat: str, rank: int, valid: bool):
        self.pattern = pat
        self.rank = rank
        self.valid = valid

    def __repr__(self):
        return 'Features(%s,%d,%s)' % (self.pattern, self.rank, self.valid)


class Motif:
    """ Motif: m-length sequences in the reads.
        position - position in read
        feature - feature with (pattern, rank, valid)
    """

    __slots__ = ('position', 'feature')
    position: int
    feature: Features

    def __init__(self, pos: int, feature: Features):
        self.position = pos
        self.feature = feature

    def __repr__(self):
        return 'Position=%s, Feature=(%s)' % (self.position, self.feature)

__Empty__ = Motif(-1, Features('', -1, False))


## ShiftScanner

In [32]:
class ShiftScanner:
    def __init__(self, space):
        assert(space.width <= 15)
        self.space = space

        global width, mask
        width = space.width
        mask = (1 << (width*2)) - 1

        self.featuresByPriority = [Features(m, p, True) for p, m in enumerate(space.byPriority)]

    """ Find all matches in the string.
        Returns an array with the matches in order, or Motif.Empty for positions
        where no valid matches were found.
    """

    def allMatches(self, read: str, tolist: bool = False):
        pos = 0
        window = 0
        while ((pos < width - 1) and pos < len(read)):
            window = ((window << 2) | charToTwobit(read[pos]))
            pos += 1

        def getMatches() -> iter:
            nonlocal pos, window
            while pos < len(read):
                window = ((window << 2) | charToTwobit(read[pos])) & mask 
                pos += 1

                priority = self.space.priorityLookUp[window]
                if(priority != -1):
                    motifPos = pos - width
                    features = self.featuresByPriority[priority]
                    yield Motif(motifPos, features)
                else:
                    yield __Empty__
    
        
        def _getMatches(pos, motif: str) -> iter:
            priority = self.space.priorityOf(motif)
            if(priority != -1):
                return Motif(pos, Features(motif, priority, True))
            return __Empty__

        if tolist:
            return [_getMatches()(read[pos:pos+width], pos) for pos in range(len(read) - (width-1))]

        # return (getMatches(pos, read[pos:pos+width]) for pos in range(len(read) - (width-1)))
        return getMatches()

## MotifSpace

In [39]:
from itertools import product
import numpy as np
class MotifSpace:
    """ MotifSpace: create the priority lookup table from which
        priority/rank of each Motif can be accessed easily
    """
    __slots__ = ('width', '_maxMotifs', '_shift', 'scanner', 'byPriority', 'priorityLookUp')
    width: int
    byPriority: np.ndarray
    scanner: ShiftScanner
    _maxMotifs: int
    _shift: int
    priorityLookUp: np.ndarray

    def __init__(self, byPriority: np.ndarray):
        self.byPriority = np.array(list(byPriority))
        self.width = len(self.byPriority[0]) 
        self.scanner = ShiftScanner(self)
        self._maxMotifs = 4 << (self.width * 2 - 2)
        self._shift = 64 - (self.width * 2)
        self.priorityLookUp = np.ndarray(self._maxMotifs, dtype=np.int64)
        
        self.priorityLookUp.fill(-1)
        for pri, motif in enumerate(self.byPriority):
            idx = self.motifToInt(motif)
            print(idx)
            self.priorityLookUp[idx] = pri
        
    def motifToInt(self, m: str) -> int:
        wrapped = _NTBitArray().encode(m)
        print(wrapped.data)
        x = wrapped.partAsLongArray(0, self.width)
        return np.uint64(x[0]) >> np.uint64(self._shift) 

    def priorityOf(self, mk):
        return self.priorityLookUp[self.motifToInt(mk)]

    def create(self, pattern: str, pos: int) -> Motif:
        return Motif(pos, Features(pattern, self.priorityOf(pattern), True))



all1mersDNA = ("A", "C", "G", "T")
all1mersRNA = ("A", "C", "G", "U")
    
def motifsOfLength(width: int, rna: bool = False) -> iter:
    bases = all1mersRNA if rna else all1mersDNA

    # def generate(prefix, length):
    #     if (length == 0):
    #         yield prefix
    #         return

    #     for base in bases:
    #         yield from generate(prefix + base, length-1)

    return (''.join(p) for p in product(bases, repeat=width))

def ofLength(w: int, rna: bool = False) -> MotifSpace:
    return using(motifsOfLength(w, rna))

def using(mers: list) -> MotifSpace:
    return MotifSpace(mers)

def fromTemplateWithValidSet(template: MotifSpace, validMers: iter) -> MotifSpace:
    validSet = set(validMers)
    return MotifSpace(filter(lambda _: _ in validSet, template.byPriority))

In [40]:
MotifSpace(motifsOfLength(4)).motifToInt('ACGT')

[0]
0
[72057594037927936]
1
[144115188075855872]
2
[216172782113783808]
3
[288230376151711744]
4
[360287970189639680]
5
[432345564227567616]
6
[504403158265495552]
7
[576460752303423488]
8
[648518346341351424]
9
[720575940379279360]
10
[792633534417207296]
11
[864691128455135232]
12
[936748722493063168]
13
[1008806316530991104]
14
[1080863910568919040]
15
[1152921504606846976]
16
[1224979098644774912]
17
[1297036692682702848]
18
[1369094286720630784]
19
[1441151880758558720]
20
[1513209474796486656]
21
[1585267068834414592]
22
[1657324662872342528]
23
[1729382256910270464]
24
[1801439850948198400]
25
[1873497444986126336]
26
[1945555039024054272]
27
[2017612633061982208]
28
[2089670227099910144]
29
[2161727821137838080]
30
[2233785415175766016]
31
[2305843009213693952]
32
[2377900603251621888]
33
[2449958197289549824]
34
[2522015791327477760]
35
[2594073385365405696]
36
[2666130979403333632]
37
[2738188573441261568]
38
[2810246167479189504]
39
[2882303761517117440]
40
[2954361355555045

27

## MotifExtracter

In [25]:
class MotifExtractor:
    __slots__ = ('scanner', 'K', 'width')
    scanner: ShiftScanner
    K: int
    width: int

    """ Extract all the best motifs from the reads
    """
    def __init__(self, space: MotifSpace, k: int):
        self.scanner = space.scanner
        self.K = k
        self.width = space.width

    """ Find all the top (best Motifs within K-length window)
    """
    def slidingTopMotifs(self, read):
        matches = self.scanner.allMatches(read)
        windowMotifs = PosRankWindow()

        pos = self.width - self.K
        for m in matches:
            windowMotifs.moveWindowAndInsert(pos, m)
            pos += 1
            yield windowMotifs.top()
                
    """ find the regions of the top motifs in reads
    """
    def regionsInRead(self, read):
        topMotifs = self.slidingTopMotifs(read)
        self.drop(topMotifs, self.K - self.width)
        
        lastMotif = next(topMotifs, None)
        
        consumed = 1
        startReg = 1
        for motif in topMotifs:
            if lastMotif == motif:
                consumed += 1
            else:
                yield lastMotif, startReg-consumed
                lastMotif = motif
                consumed = 1

            startReg += 1

        # for the last motif
        yield lastMotif, startReg - consumed

    """ Return all the Super-mers
    """
    def splitRead(self, read):
        if (len(read) < self.K):
            return
        
        readByReg = self.regionsInRead(read)

        prev = next(readByReg, None)
        while readByReg:
            b1 = prev
            b2 = next(readByReg, None)
            
            if b2:
                yield b1[0], read[b1[1]: b2[1] + (self.K - 1)]
                prev = b2
            else:
                yield b1[0], read[b1[1]:]
                break

    @staticmethod
    def drop(itr, n):
        j = 0
        try:
            while (j < n):
                next(itr)
                j += 1
        except StopIteration:
            return


## MotifCounter

In [26]:
class MotifCounter:
    """ Main frequency counter
    """
    def __init__(self, space: MotifSpace):
        sizeOfCounter = len(space.byPriority)
        self.countArr = np.ndarray(sizeOfCounter, dtype=np.int64)

    """ Increment the count of the corresponding Motif
    """
    def increment(self, m: Motif):
        rank = m.feature.rank
        self.countArr[rank] += 1

    def motifsWithCounts(self, space: MotifSpace):
        return zip(space.byPriority, self.countArr)

    def _toSpaceByFrequency(self, counts: list((str, int))):
        # sorting first by frequency and then sorting by lexicographically
        c = map(lambda _: _[0], sorted(counts, key=lambda _: (_[1], _[0])))
        return MotifSpace(c)

    """ Make a new space to with priority based on frequency and lexicographically ordering
    """
    def toSpaceByFrequency(self, oldSpace: MotifSpace):
        pairs = self.motifsWithCounts(oldSpace)
        return self._toSpaceByFrequency(pairs)
        


## MotifCountingScanner

In [27]:
class MotifCountingScanner:
    """ Looks for the motifs and find the it's occurences
        in the whole dataset
    """
    def __init__(self, space: MotifSpace):
        self.scanner = space.scanner
    
    def scanRead(self, counter: MotifCounter, read):
        for m in self.scanner.allMatches(read):
            if m.feature.valid:
                counter.increment(m)

    def scanGroup(self, counter: MotifCounter, rs):
        for r in rs:
            self.scanRead(counter, r)


## PosRankWindow

In [28]:
class PositionNode:
    """ PosiitonNode: to track and find the best Motifs
        lesser the rank higher is the priority
    """
    __slots__ = ['prevPos', 'nextPos']
    prevPos: ...
    nextPos: ...
    
    def __init__(self, p = '_', n = '_'):
        self.prevPos = p
        self.nextPos = n
        
    def removeNode(self):
        temp = self.nextPos
        self.prevPos.nextPos = self.nextPos
        self.nextPos.prevPos = self.prevPos
        
        del self
        return temp
        
    def linkPos(self, before, after):
        before.nextPos = self
        self.prevPos = before
        self.nextPos = after
        after.prevPos = self


class MotifContainer(PositionNode):
    """ MotifContainer: Store the Motif and is the Node in PosRankWindow (doubly linked-list)
        pos - position of the Motif in the read
        motif - Motif 
        rank - rank of the Motif
    """
    __slots__ = ['pos', 'motif', 'rank']
    pos: int
    motif: Motif
    rank: int
    
    def __init__(self, motif: Motif):
        self.pos = motif.position
        self.rank = motif.feature.rank
        self.motif = motif
    
    def dropUntilPosition(self, pos: int):
        if self.pos < pos:
            self = self.removeNode()
            self.dropUntilPosition(pos)
            
    def __repr__(self):
        return '[%s, %s]' % (self.pos, self.motif)

        
class PosRankWindow:
    """ PosRankWindow: A doubly linked-list that makes easier to find the best Motif within a
                         window of k-length
    """
    start: PositionNode = PositionNode()
    end: PositionNode = PositionNode()
    
    def __init__(self):
        # None <= start <=> end => None
        self.start.nextPos = self.end
        self.start.prevPos = None
        self.end.prevPos = self.start
        self.end.nextPos = None
    
    """ Moves the window and insert the new Node to the right and removes from the left 
    """
    def moveWindowAndInsert(self, pos: int, insertRight):
        new_node = MotifContainer(insertRight)
        if new_node.motif.feature.valid:
            self.appendMonotonic(new_node, self.end)
        
        if self.start.nextPos != self.end and self.end.prevPos != self.start:
            self.start.nextPos.dropUntilPosition(pos)
    
    """ Potentially insert the Motifs to the correct location
    """
    def appendMonotonic(self, insertNode, search):
        # if this(start <=> end) is not the case
        if search.prevPos != self.start:
            if insertNode.rank < search.prevPos.rank:
                self.appendMonotonic(insertNode,search.prevPos)
            else:
                insertNode.linkPos(search.prevPos, self.end)
        else:
            insertNode.linkPos(self.start, self.end)
    
    """ Return the best motif in each k-length window
    """
    def top(self):
        return __Empty__ if self.start.nextPos==self.end else self.start.nextPos.motif 
    

    """ For the testing purpose to check what is happening in the window
    """
    def showWindow(self):
        temp = self.start.nextPos
        while temp != self.end:
            print('window-pos=',temp.pos, temp.motif)
            temp = temp.nextPos

## Col

In [8]:
class Col:
    """ Y - YELLOW | B - BOLD | G - GREEN | U - UNDERLINE
        V - PURPLE | C - CYAN | W - WHITE | R - RED | GR - GREY
    """
    V = '\033[95m'
    GR = '\u001b[30;1m'
    C = '\033[38;2;0;200;255m'
    G = '\033[92m'
    Y = '\033[93m'
    R = '\033[91m'
    W = '\033[0m'
    B = '\033[1m'
    U = '\033[4m' 

## CoreConf

In [9]:
import os

class DiscountCustomError(Exception):
    def __init__(self, msg, arg, req):
        super().__init__(msg, arg, req)
        self.msg = msg
        self.req = req
        self.has = arg
        
    def __str__(self):
        return Col.B + Col.R + self.msg + Col.G + " | " + Col.C\
                             + self.has + Col.V + u" \u2550"u"\u2550> " + Col.C\
                             + self.req + Col.W

def verify(args):
    print(Col.G, 'Verifying the input...', Col.W)
    """ Check for the fasta file format """
    
    if args.count:
        if not args.f.endswith('.txt'):
            raise DiscountCustomError("FileFormatError : required .txt file", args.f.split('/')[-1], "XXX.txt")
        else:
            try:
                os.mkdir(args.count)
            except OSError:
                pass
    else:
        if not args.f.endswith('.fasta'):
            raise DiscountCustomError("FileFormatError : required .fasta file", args.f, "XXX.fasta")
    
    """ Check for ordering : if universal frequency minimizer must be entered """
    if args.o == 'ufreq' and not args.minimizers:
        raise DiscountCustomError("UniversalFrequencyOrdering : required -minimizers", args.o, "-minimizers")

    """ Check if the given output dir exist or not """
    if args.output and not os.path.isdir(args.output.split('/')[0]):
        raise DiscountCustomError("No such file or directory:", '', args.output)

class CoreConf:
    __slots__ = ['K','WIDTH','MINIMIZERS','DATASET','ORDER','TEMPLATESPACE', 'OUTPUT', 'COUNT']
    K: int
    WIDTH: int
    MINIMIZERS: str
    DATASET: str
    ORDER: str
    TEMPLATESPACE: MotifSpace
    OUTPUT: str
    COUNT: str
    
    def __init__(self, args):
        verify(args)
        print(Col.Y, 'Verification done successfully')
        
        self.K = args.k
        self.DATASET = args.f
        self.WIDTH = args.m if args.m else 10
        self.MINIMIZERS = args.minimizers
        self.ORDER = args.o
        self.OUTPUT = args.output  
        print(Col.V, 'Executing...', Col.W)    
        self.TEMPLATESPACE = MotifSpace(motifsOfLength(width=self.WIDTH))
        
        print(self.K, self.WIDTH, self.DATASET, self.MINIMIZERS, self.ORDER)

## ReadSplitDemo

In [10]:
class ReadSplitConf(CoreConf):
    def __init__(self, args):
        super().__init__(args)

    def getInputSequences(self) -> iter:
        with open(self.DATASET) as f:
            return map(lambda _: re.split(r"[^ACGTU]+", _.rstrip()), 
                filter(lambda _: not _.startswith('>'), f.readlines()))
        
    def getFrequencySpace(self, validMotifs) -> MotifSpace:
        inputdata = self.getInputSequences()
        template = fromTemplateWithValidSet(self.TEMPLATESPACE, validMotifs)
        counter = MotifCounter(template)
        scanner = MotifCountingScanner(template)
        scanner.scanGroup(counter, inputdata)
        return counter.toSpaceByFrequency(template)
  
    def getSplitter(self) -> MotifExtractor:
        def use():
            with open(self.MINIMIZERS) as f:
                return (_.rstrip() for _ in f.readlines())

        template = self.TEMPLATESPACE
        validMotifs = use() if self.MINIMIZERS else template.byPriority
        
        useSpace = template if self.ORDER == 'lex' else self.getFrequencySpace(validMotifs)

        return MotifExtractor(useSpace, self.K)

def printSup(s, k):
    print(f'{s[0].feature.pattern} (pos {s[0].position}, rank {s[0].feature.rank}, len {len(s[1])-k+1} k-mers)', end=' ')
    before, after = s[1].split(s[0].feature.pattern, 1)
    print(f'{before}{Col.V}{s[0].feature.pattern}{Col.W}{after}')


def readSplitDemo(args):
    print(args)
    print(Col.C, 'Running Discount...')
    
    conf = ReadSplitConf(args)
        
    spl = conf.getSplitter()
    if conf.OUTPUT:
        with open(conf.OUTPUT, 'w+') as f:
            for r in conf.getInputSequences():
#                 print('Read:', r)
                for s in spl.splitRead(r):
                    f.write('{0}\t{1}\n'.format(s[0].feature.pattern, s[1]))
#                     printSup(s, spl.K)
    else:
        for r in conf.getInputSequences():
            print('Read :', r)
            for s in spl.splitRead(r):
                printSup(s, spl.K)

    

## Main

In [11]:
import argparse

class CustomArgparser(argparse.ArgumentParser):
    def error(self, message):
        print(Col.R + Col.B + 'ERROR : %s\n' % message)
        print(Col.W + 'For more details run :' + Col.Y + ' discount ' + Col.GR + '-h' + Col.W)
        exit()


def main(args):
    # print(sys.argv)    
    parser = CustomArgparser(prog="discount", description="\n\tDiscountPy : A k-mer counting tool")
    count = parser.add_argument_group('Count k-mers')
    count.add_argument('--count', metavar='', type=str, help='Generate k-mers counted file. (Input file(.txt) shoud be sorted by minimizers with corresponding super-mers)')
    req = parser.add_argument_group('Required Arguments')
    parser.version = Col.G + 'DiscountPy version ' + Col.C + __version__ + Col.W
    
    req.add_argument("-k", metavar='', type=int, help="Length of the k-mers", required=True)
    parser.add_argument("-m", metavar='', type=int, help="Width of the minimizers (default 10)", default=10)
    req.add_argument("-f", metavar='', type=str, help="Dataset (.fasta)", required=True)
    parser.add_argument("-o", type=str, choices=["lex", "freq"], default="freq",
                              help="Ordering {lex | lexicographic, freq | frequency} (default freq)")
    parser.add_argument("--minimizers", metavar="", type=str, help="Valid minimizers sets")
    parser.add_argument('--output', metavar='', type=str, help='Generates output of Super-mers with minimizers')
    parser.add_argument("-v", '--version', action='version', help="Version of the tool")

    readSplitDemo(parser.parse_args(args))

In [4]:
import cProfile
import line_profiler
%load_ext line_profiler

In [13]:
%%time

file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/data/1M.fasta' 
pasha = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/PASHA/pasha_all_28_10.txt'
out = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output' + '/1M_ufreq.txt'
args = ['-k', '28', '-m', '10', '-f', file, '-o', 'freq', '--minimizers', pasha, '--output', out]

%prun -D 100k_ufreq.prof main(args)

Namespace(count=None, k=28, m=10, f='C:/Users/umesh/Desktop/Discount-In-Python/Discount/data/1M.fasta', o='freq', minimizers='C:/Users/umesh/Desktop/Discount-In-Python/Discount/PASHA/pasha_all_28_10.txt', output='C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/1M_ufreq.txt')
 Running Discount...
 Verifying the input... 
 Verification done successfully
 Executing... 
28 10 C:/Users/umesh/Desktop/Discount-In-Python/Discount/data/1M.fasta C:/Users/umesh/Desktop/Discount-In-Python/Discount/PASHA/pasha_all_28_10.txt freq
 
*** Profile stats marshalled to file '100k_ufreq.prof'. 
Wall time: 1h 1min 30s


### version 1

In [10]:
# version 1
import os
from itertools import groupby, chain

def flatten(sm):
    return chain.from_iterable(sm)

def sliding(supermer: str, k: int):
    return (supermer[i : i + k] for i in range(len(supermer) - k + 1))

def countKmers(k: int, inputFile: str, bucket: str):
    if not inputFile.endswith('.txt'):
        raise DiscountCustomError("FileFormatError : required .txt file", inputFile.split('/')[-1], "XXX.txt")
    else:
        try:
            path = bucket.rsplit('/', 1)[0]
            print(path)
            os.mkdir(bucket.rsplit('/', 1)[0])
        except OSError:
            pass
    
    with open(inputFile) as f:
        lines = map(lambda _: _.rstrip('\n').split('\t'), f)
        
        print(bucket)
        with open(bucket, 'w+') as f:
            for minimizer, thisBin in groupby(lines, lambda _: _[0]):
                supermersThisBin = map(lambda _: _[1], thisBin)

                slide = map(lambda sm: sliding(sm, k), supermersThisBin)
                flat = flatten(slide)
                kmers = iter(sorted(flat))

                # kmers = sorted(flat)
                # print(minimizer, kmers)

                last = next(kmers)
                count = 1

                for _ in kmers:
                    if _ == last:
                        count += 1
                    else:
                        f.write(f'{last}\t{count}\n')
                        last = _
                        count = 1

                f.write(f'{last}\t{count}\n')

In [11]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_freq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq/kmers.txt'
k = 28

%lprun -T 100k_freq.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq
C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq/kmers.txt

*** Profile printout saved to text file '100k_freq.txt'. 
Wall time: 3min 45s


### version 2

In [18]:
# version 2

import os
from more_itertools import flatten, peekable, run_length, bucket as buk

def sliding(supermer: str, k: int):
    return (supermer[i : i + k] for i in range(len(supermer) - k + 1))

def takewhile(lines, m):
    while lines and lines.peek()[0] == m:
        yield next(lines)[1]

def countKmers(k: int, inputFile: str, bucket: str):
    if not inputFile.endswith('.txt'):
        raise DiscountCustomError("FileFormatError : required .txt file", inputFile.split('/')[-1], "XXX.txt")
    else:
        try:
            path = bucket.rsplit('/', 1)[0]
            print(path)
            os.mkdir(bucket.rsplit('/', 1)[0])
        except OSError:
            pass
    
    with open(inputFile, 'r') as f:
        lines = peekable(map(lambda _: _.rstrip('\n').split('\t'), f))
#         lines = buk(map(lambda _: _.rstrip('\n').split('\t'), f), lambda _: _[0])
#         for _ in lines:
#             print(list(lines[_]))
#         print(list(lines))

        result = []
        while lines:
            minimizer = lines.peek()[0]
            supermersThisBin = takewhile(lines, minimizer)

            kmers = peekable(sorted(flatten(map(lambda _: sliding(_, k), supermersThisBin))))
            
            count = 0
            lastKmer = kmers.peek()
            while kmers:
                if kmers.peek() == lastKmer:
                    count += 1
                    next(kmers)
                else:
                    result.append('%s\t%s'%(lastKmer, count))
                    count = 1
                    lastKmer = next(kmers)
                    
            result.append('%s\t%s'%(lastKmer, count))

        open(bucket, 'w').write(''.join('%s\n'%(_) for _ in result))

In [19]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_freq.txt' 
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq/kmers2.txt'
k = 28

%lprun -T 100k_freq2.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq

*** Profile printout saved to text file '100k_freq2.txt'. 
Wall time: 4min 49s


### version 3

In [21]:
# version 3
import os
from itertools import groupby, chain

def flatten(sm):
    return chain.from_iterable(sm)

def sliding(supermer: str, k: int):
    return (supermer[i : i + k] for i in range(len(supermer) - k + 1))

def countKmers(k: int, inputFile: str, bucket: str):
    if not inputFile.endswith('.txt'):
        raise DiscountCustomError("FileFormatError : required .txt file", inputFile.split('/')[-1], "XXX.txt")
    else:
        try:
            path = bucket.rsplit('/', 1)[0]
            print(path)
            os.mkdir(bucket.rsplit('/', 1)[0])
        except OSError:
            pass
    
    with open(inputFile) as f:
        lines = map(lambda _: _.rstrip('\n').split('\t'), f)
        
        result = []
        for minimizer, thisBin in groupby(lines, lambda _: _[0]):
            supermersThisBin = map(lambda _: _[1], thisBin)

            slide = map(lambda sm: sliding(sm, k), supermersThisBin)
            flat = flatten(slide)
            kmers = iter(sorted(flat))

            # kmers = sorted(flat)
            # print(minimizer, kmers)

            last = next(kmers)
            count = 1

            for _ in kmers:
                if _ == last:
                    count += 1
                else:
                    result.append('%s\t%s'%(last, count))
                    last = _
                    count = 1

            result.append('%s\t%s'%(last, count))
        
        open(bucket, 'w').write(''.join('%s\n'%(_) for _ in result))
        

In [22]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_freq.txt' 
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq/kmers3.txt'
k = 28

%lprun -T 100k_freq3.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_100k_freq

*** Profile printout saved to text file '100k_freq3.txt'. 
Wall time: 4min


### version 4

In [2]:
# version 4
import os
from more_itertools import peekable
from collections import Counter

def countKmers(k: int, inputFile: str, bucket: str):
    if not inputFile.endswith('.txt'):
        raise DiscountCustomError("FileFormatError : required .txt file", inputFile.split('/')[-1], "XXX.txt")
    else:
        try:
            path = bucket.rsplit('/', 1)[0]
            print(path)
            os.mkdir(bucket.rsplit('/', 1)[0])
        except OSError:
            pass
    
    def takeWhile(lines, m):
        _thisBin = Counter()

        while lines and lines.peek()[0] == m:
            _thisBin.update(next(lines)[1])

        return _thisBin

    def sliding(supermer):
        return [supermer[i : i + k] for i in range(len(supermer) - k + 1)]

    def kmer_fun(x):
        return x[0], sliding(x[1])
        
    with open(inputFile) as f:
        lines = peekable(map(lambda _: kmer_fun(_.rstrip('\n').split('\t')), f))
        
        with open(bucket, 'w+') as demo:
            while lines:
                minimizer = lines.peek()[0]
                supermersThisBin = takeWhile(lines, minimizer)
                demo.writelines(['%s\t%s\n'%(_, supermersThisBin[_]) for _ in supermersThisBin])

In [161]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_freq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_demo/freq.txt'
k = 28

%lprun -T freq.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_demo

*** Profile printout saved to text file 'freq.txt'. 
Wall time: 1min 53s


In [162]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_ufreq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_demo/ufreq.txt'
k = 28

%lprun -T ufreq.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_demo

*** Profile printout saved to text file 'ufreq.txt'. 
Wall time: 1min 22s


In [164]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_freq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_prof/freq.txt'
k = 28

%prun -D freq.prof countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_prof
 
*** Profile stats marshalled to file 'freq.prof'. 
Wall time: 1min 22s


In [165]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_100k_ufreq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_prof/ufreq.txt'
k = 28

%prun -D ufreq.prof countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_prof
 
*** Profile stats marshalled to file 'ufreq.prof'. 
Wall time: 1min 2s


In [5]:
%%time
file = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/output/sort_1M_ufreq.txt'
bucket = 'C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_1M/ufreq.txt'
k = 28

%lprun -T ufreq.txt -f countKmers countKmers(k, file, bucket)

C:/Users/umesh/Desktop/Discount-In-Python/Discount/bucket_1M

*** Profile printout saved to text file 'ufreq.txt'. 
Wall time: 9min 57s


## BitRepresentation

In [12]:
A: bytes = 0x0
C: bytes = 0x1
G: bytes = 0x2
T: bytes = 0x3
U: bytes = 0x3 # In RNA
    
""" Convert a single BP from twobit representation to string representation. """
def twobitToChar(byte: bytes):
    switcher = {
        0 : 'A',
        1 : 'C',
        2 : 'G',
        3 : 'T'
    }

    return switcher.get(byte, "Invalid Value")

""" Convert a single nucleotide from string (char) representation to "twobit" representation. """
def charToTwobit(char: str) -> bytes:
    switcher = {
        'A' : A,
        'C' : C,
        'G' : G,
        'T' : T,
        'U' : U
    }

    return switcher.get(char, "Invalid Nucleotide")

twobits = (A, C, T, G)

""" Convert a single byte to the "ACTG" format (a 4 letter string) """
def byteToQuadCompute(byte: bytes):
    res = ""
    for i in range(4):
        ptn = (byte >> ((3 - i) * 2)) & 0x3
        char = twobitToChar(ptn)
        res += char

    return res

""" Map a single byte to a quad-string for unpacking.
    Precomputed lookup array."""
byteToQuadLookup = [byteToQuadCompute(i) for i in range(256)]

""" Complement of a single BP. """
def complementOne(byte: bytes) -> int:
    return complement(byte) & 0x3

""" Complement of a number of BPs packed in a byte. """
def complement(byte: bytes) -> bytes:
    return (byte ^ 0xff)

""" Map a quad-string (four letters) to an encoded byte """
def quadToByte(quad: str) -> bytes:
    return _quadToByte(quad, 0)


""" Unpack a byte to a 4-character string (quad). """
def byteToQuad(byte: bytes):
    return byteToQuadLookup[byte]

""" Convert an NT quad (string of length 4) to encoded byte form. 
    The string will be padded on the right with 'A' if it's too short. """
def _quadToByte(quad: str, offset: int) -> np.int64:
    res = 0
    i = offset
    end = offset + 4
    while (i < end):
        c = 'A' if(i >= len(quad)) else quad[i]
        twobits = charToTwobit(c)
        res = twobits if i==0 else (res << 2) | twobits
        # print(res, format(res, '064b'))
        i += 1
    
    return np.int64(res)
    
""" Convert a string to an array of quads. """
def stringToBytes(bps: str) -> list:
    rsize = (len(bps) - 1) // 4
    r = [_quadToByte(bps, i*4) for i in range(rsize+1)]
    
    return r

""" Convert a byte array of quads to a string. The length of the resulting string must be supplied. """
def bytesToString(byteArr: bytearray, builder: str, offset: int, size: int) -> str:
    # print(byteArr)
    startByte = offset // 4

    i = startByte
    while (i < len(byteArr)):
        if (len(builder) < size):
            if (i == startByte):
                of = offset % 4
                builder += byteToQuad(byteArr[i])[of : of + 4]
            else:
                builder += byteToQuad(byteArr[i])
        i += 1

    return builder[:size]

## NTBitArray

In [13]:
class _NTBitArray:
    def encode(self, data: str) -> ...:
        buf = self.longBuffer(len(data))
        longIdx = 0
        quadOffset = 0
        while longIdx < len(buf):
            quadIdx = 0
            qshift = 56
            x = 0
            while quadIdx < 8 and quadOffset < len(data):
                quad = _quadToByte(data, quadOffset)
                x |= (quad & 255) << qshift

                qshift -= 8
                quadIdx += 1
                quadOffset += 4
            
            buf[longIdx] = x
            longIdx += 1
        
        return ZeroNTBitArray(buf, len(data))

    def longBuffer(self, size: int) -> np.array([], dtype=np.int64):
        numLongs = (size >> 5) if(size % 32 == 0) else (size >> 5) + 1
        return np.zeros(numLongs, dtype=np.int64)

    def longsToString(self, data: np.array([], np.int64), offset: int, size: int) -> str:
        sb = ''
        byte = (size // 4) if (size % 32 == 0) else (size // 4 + 8)
        return self._longsToString(byte, sb, data, offset, size)

    """
    Optimised version for repeated calls - avoids allocating a new buffer each time
    """
    def _longsToString(self, byte: int, builder: str, data: np.array([], np.int64), offset: int, size: int) -> str:
        data.newbyteorder().byteswap(inplace=True)
        buffer = bytearray(data)
        data.newbyteorder().byteswap(inplace=True)
        return bytesToString(buffer, builder, offset, size)

    
class NTBitArray(_NTBitArray):
    """
    A bit-packed array of nucleotides, where each letter is represented by two bits
    """
    __slots__ = ['data', 'offset', 'size', 'toString']

    """Array of longs, each storing up to 16 nts, with optional padding at the end."""
    data: np.array([], dtype=np.int64)

    """Offset into the array where data starts (in NTs)"""
    offset: int

    """Size of the data represented (in NTs)"""
    size: int

    toString: str

    def __init__(self, data: np.array([], dtype=np.int64), offset: int, size: int):
        self.data = data
        self.offset = offset
        self.size = size
        self.toString = self.longsToString(data, offset, size)

    """Construct a new NTBitArray from a subsequence of this one."""
    def slice(self, start: int, length: int) -> ...:
        return self.OffsetNTBitArray(self.data, start, length)

    """
    Obtain the (twobit) NT at a given offset.
    Only the lowest two bits of the byte are valid. The others will be zeroed out.
    """
    def apply(self, pos: int) -> bytes:
        truePos = self.offset + pos
        lng = truePos // 32
        lval = self.data[lng]
        localOffset = truePos % 32

        return np.int64((lval >> (2 * (31 - localOffset))) & 0x3).tobytes()

    """
    Test the orientation of a slice of this buffer.
    :param pos: Start position
    :param size: Length of slice (must be an odd number)
    :return: True iff this slice has forward orientation.
    """
    def sliceIsForwardOrientation(self, pos: int, size: int) -> bool:
        st = pos
        end = pos + size - 1
        while (st < end):
            a = self.apply(st)
            b = complementOne(self.apply(end))
            if (a < b): return True
            if (a > b): return False

            st += 1
            end -= 1

        # Here, st == end
        # Resolve a nearly palindromic case, such as: AACTT whose r.c. is AAGTT
        return self.apply(st) < G

    """
    Obtain all k-mers from this buffer.
    :param k: int
    :param onlyForwardOrientation: bool = If this flag is true, only k-mers with forward orientation will be returned.
    :return: forward orientation 
    """
    def kmers(self, k: int, onlyForwardOrientation: bool = False) -> iter:
        return map(lambda i: slice(i, k), 
            filter(lambda i: (not onlyForwardOrientation) or self.sliceIsForwardOrientation(i, k), 
            (i for i in range(self.offset, self.size - k + 1))))

    """
    Obtain all k-mers from this buffer, packed as Array[Long].
    :param k
    :param onlyForwardOrientation If this flag is true, only k-mers with forward orientation will be returned.
    :return
    """
    def kmersAsLongArrays(self, k: int, onlyForwardOrientation: bool = False) -> iter:
        return filter(lambda _: _ != None, self.__kmersAsLongArraysOrientationMatch__(k, onlyForwardOrientation))

    """
    Create a long array representing a subsequence of this bpbuffer.
    :param offset
    :param size
    :return
    """
    def partAsLongArray(self, offset: int, size: int) -> np.array([], dtype=np.int64):
        buf = self.longBuffer(size)
        self.copyPartAsLongArray(buf, offset, size)
        return buf

    def copyPartAsLongArray(self, writeTo: np.array([], dtype=np.int64), offset: int, size: int):
        shiftAmt = (offset % 32) * 2

        finalKeepBits = 64 if (size % 32 == 0) else (size % 32) * 2
        finalLongMask = np.int64(-1) << (64 - finalKeepBits)

        numLongs = (size >> 5) if (size % 32 == 0) else (size >> 5) + 1
        sourceLongs = len(self.data)
        i = 0
        read = offset // 32
        while (i < numLongs):
            x = self.data[read] << shiftAmt
            if (read < sourceLongs - 1 and shiftAmt > 0):
                x = x | (self.data[read + 1] >> (64 - shiftAmt))
        
            if (i == numLongs - 1):
                x = x & finalLongMask

            writeTo[i] = x
            read += 1
            i += 1


class OffsetNTBitArray(NTBitArray):
    def __init__(self, data: np.array([], dtype=np.int64), offset: int, size: int):
        self.data = data
        self.offset = offset
        self.size = size

class ZeroNTBitArray(NTBitArray):
    def __init__(self, data: np.array([], dtype=np.int64), size: int):
        super().__init__(data, 0, size)

In [29]:
%%time

space = ofLength(10, False)

Wall time: 51.2 s


In [30]:
space.byPriority

array(['AAAAAAAAAA', 'AAAAAAAAAC', 'AAAAAAAAAG', ..., 'TTTTTTTTTC',
       'TTTTTTTTTG', 'TTTTTTTTTT'], dtype='<U10')

In [ ]:
NTBitArray().copyPartAsLongArray()

In [33]:
%%time

m = space.scanner.allMatches("CCCGAATACAAAATTGGTCTTTGGCGAAACGATTGGAAACCCGGCGCTTGGCGTTCTGGATTTCGAAAAGTTCTCGAAGCTTGCCAAGGAATTCGATGTGC")
print(list(m))

0 <class 'int'> 1 <class 'int'>
1 <class 'int'> 1 <class 'int'>
5 <class 'int'> 1 <class 'int'>
21 <class 'int'> 2 <class 'int'>
86 <class 'int'> 0 <class 'int'>
344 <class 'int'> 0 <class 'int'>
1376 <class 'int'> 3 <class 'int'>
5507 <class 'int'> 0 <class 'int'>
22028 <class 'int'> 1 <class 'int'>
[Position=0, Feature=(Features(CCCGAATACA,352452,True)), Position=1, Feature=(Features(CCGAATACAA,361232,True)), Position=2, Feature=(Features(CGAATACAAA,396352,True)), Position=3, Feature=(Features(GAATACAAAA,536832,True)), Position=4, Feature=(Features(AATACAAAAT,50179,True)), Position=5, Feature=(Features(ATACAAAATT,200719,True)), Position=6, Feature=(Features(TACAAAATTG,802878,True)), Position=7, Feature=(Features(ACAAAATTGG,65786,True)), Position=8, Feature=(Features(CAAAATTGGT,263147,True)), Position=9, Feature=(Features(AAAATTGGTC,4013,True)), Position=10, Feature=(Features(AAATTGGTCT,16055,True)), Position=11, Feature=(Features(AATTGGTCTT,64223,True)), Position=12, Feature=(Feature